# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc4949337c0>
├── 'a' --> tensor([[ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544]])
└── 'x' --> <FastTreeValue 0x7fc4949336a0>
    └── 'c' --> tensor([[ 1.0421,  0.2409,  1.1599,  0.7950],
                        [ 1.5254, -0.6909, -0.1304,  1.7306],
                        [ 0.3316,  2.7164, -0.0161, -0.2422]])

In [4]:
t.a

tensor([[ 1.3894, -2.4347, -0.6965],
        [-0.4570, -0.9840,  0.0544]])

In [5]:
%timeit t.a

63.8 ns ± 0.0762 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc4949337c0>
├── 'a' --> tensor([[-0.5429,  0.1682,  0.4332],
│                   [-0.3199, -1.3713,  0.0855]])
└── 'x' --> <FastTreeValue 0x7fc4949336a0>
    └── 'c' --> tensor([[ 1.0421,  0.2409,  1.1599,  0.7950],
                        [ 1.5254, -0.6909, -0.1304,  1.7306],
                        [ 0.3316,  2.7164, -0.0161, -0.2422]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.0408 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3894, -2.4347, -0.6965],
               [-0.4570, -0.9840,  0.0544]]),
    x: Batch(
           c: tensor([[ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3894, -2.4347, -0.6965],
        [-0.4570, -0.9840,  0.0544]])

In [11]:
%timeit b.a

58.7 ns ± 0.0264 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5327,  0.6855, -1.4066],
               [ 2.1498, -0.1980,  0.3233]]),
    x: Batch(
           c: tensor([[ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.058 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.219 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 24.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc494933dc0>
├── 'a' --> tensor([[[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]],
│           
│                   [[ 1.3894, -2.4347, -0.6965],
│                    [-0.4570, -0.9840,  0.0544]]])
└── 'x' --> <FastTreeValue 0x7fc494933cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 64.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc3bc70eeb0>
├── 'a' --> tensor([[ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544],
│                   [ 1.3894, -2.4347, -0.6965],
│                   [-0.4570, -0.9840,  0.0544]])
└── 'x' --> <FastTreeValue 0x7fc3bc70ef40>
    └── 'c' --> tensor([[ 1.0421,  0.2409,  1.1599,  0.7950],
                        [ 1.5254, -0.6909, -0.1304,  1.7306],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 33.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.7 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0421,  0.2409,  1.1599,  0.7950],
                       [ 1.5254, -0.6909, -0.1304,  1.7306],
                       [ 0.3316,  2.7164, -0.0161, -0.2422]],
              
                      [[ 1.0421,  0.2409,  1.1599,  0.7950],
                       [ 1.5254, -0.6909, -0.1304,  1.7306],
                       [ 0.3316,  2.7164, -0.0161, -0.2422]],
              
                      [[ 1.0421,  0.2409,  1.1599,  0.7950],
                       [ 1.5254, -0.6909, -0.1304,  1.7306],
                       [ 0.3316,  2.7164, -0.0161, -0.2422]],
              
                      [[ 1.0421,  0.2409,  1.1599,  0.7950],
                       [ 1.5254, -0.6909, -0.1304,  1.7306],
                       [ 0.3316,  2.7164, -0.0161, -0.2422]],
              
                      [[ 1.0421,  0.2409,  1.1599,  0.7950],
                       [ 1.5254, -0.6909, -0.1304,  1.7306],
                       [ 0.3316,  2.7164, -0.0161, -0.2422]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422],
                      [ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422],
                      [ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422],
                      [ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422],
                      [ 1.0421,  0.2409,  1.1599,  0.7950],
                      [ 1.5254, -0.6909, -0.1304,  1.7306],
                      [ 0.3316,  2.7164, -0.0161, -0.2422],
                      [ 1.0421,  0.2409,  1.1599,  0.7950],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

367 µs ± 67.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
